In [8]:
# prompt: need to install pytorch 1.7.1+ cu110
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html
#!pip install torch==1.11.0+cu102 torchvision==0.12.0+cu102 torchaudio==0.11.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [13]:
import os
import torch
print("PyTorch version:", torch.__version__)
print("CUDA is available:", torch.cuda.is_available())


PyTorch version: 1.8.0+cu111
CUDA is available: True


In [10]:
!pip install keras-tuner==1.4.7



In [11]:
!nvidia-smi


Sun Apr  7 23:22:20 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.147.05   Driver Version: 525.147.05   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  Off |
|  0%   43C    P8    11W / 450W |   1539MiB / 24564MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:
!pip install --upgrade keras-tuner

In [32]:
# Import necessary libraries and modules
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam, Adadelta, Adamax
from tensorflow.keras.callbacks import EarlyStopping
from kerastuner import HyperModel, HyperParameters, Tuner
from keras_tuner.tuners import GridSearch
from tensorflow.keras.layers import Activation

# Set up image dimensions
img_height, img_width = 224, 224

# Set up the directories
train_dir = '/home/islamsa/Documents/OwlDetection/train'
validation_dir = '/home/islamsa/Documents/OwlDetection/valid'
test_dir = '/home/islamsa/Documents/OwlDetection/test'

# Data augmentation configuration for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Note: validation and test data should not be augmented
test_datagen = ImageDataGenerator(rescale=1./255)

# Data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    class_mode='categorical',
    batch_size=64
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_height, img_width),
    class_mode='categorical',
    batch_size=64
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    class_mode='categorical',
    shuffle=False
)

num_classes = train_generator.num_classes

class MyHyperModel(HyperModel):
    def __init__(self, img_height, img_width, num_classes):
        self.img_height = img_height
        self.img_width = img_width
        self.num_classes = num_classes

    def build(self, hp):
        model = Sequential()

        # Optimizing the number of convolutional layers
        for i in range(hp.Int('num_layers', 2, 4)):  # Number of layers is between 2 and 4
            model.add(Conv2D(hp.Choice(f'filters_{i}', [32, 64, 128, 256]),
                             (3, 3), padding='same',
                             input_shape=(self.img_height, self.img_width, 3)))
            model.add(Activation(hp.Choice(f'activation_{i}', ['relu', 'sigmoid', 'tanh'])))
            model.add(MaxPooling2D(pool_size=(2, 2)))
            model.add(Dropout(0.5))

        model.add(Flatten())

        model.add(Dense(256, kernel_regularizer=regularizers.l2(0.01)))
        model.add(Activation('relu'))
        model.add(Dropout(0.5))
        model.add(Dense(self.num_classes, activation='softmax'))

        # Tuner for optimizer
        hp_optimizer = hp.Choice('optimizer', ['adam', 'adadelta', 'adamax'])
        if hp_optimizer == 'adam':
            optimizer = Adam(learning_rate=hp.Choice('learning_rate', [0.001, 0.0005, 0.0001, 0.00001]))
        elif hp_optimizer == 'adadelta':
            optimizer = Adadelta(learning_rate=hp.Choice('learning_rate', [0.001, 0.0005, 0.0001, 0.00001]))
        else:  # hp_optimizer == 'adamax'
            optimizer = Adamax(learning_rate=hp.Choice('learning_rate', [0.001, 0.0005, 0.0001, 0.00001]))

        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

        return model

hypermodel = MyHyperModel(img_height, img_width, num_classes)

tuner = GridSearch(
    hypermodel,
    objective='val_accuracy',
    max_trials=5,  # Adjust number of trials
    executions_per_trial=1,
    directory='/home/islamsa/Documents/OwlDetection/hyperparameter_tuning',
    project_name='owl_detection_tuning'
)

# Define the EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)

# Start search
tuner.search(
    train_generator,
    epochs=150,
    validation_data=validation_generator,
    callbacks=[early_stopping],  # Early stopping callback
    verbose=1  # Detailed logs for each trial
)

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters()[0]

print(f"The optimal number of layers is {best_hps.get('num_layers')}.")
print(f"The optimal filters per layer are {', '.join(str(best_hps.get(f'filters_{i}')) for i in range(best_hps.get('num_layers')))}.")

# Check if each layer exists before accessing its activation function
activations = []
for i in range(best_hps.get('num_layers')):
    activation_i = best_hps.get(f'activation_{i}')
    if activation_i is not None:
        activations.append(activation_i)
print(f"The optimal activation functions per layer are {', '.join(activations)}.")

print(f"The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.")

print(f"The optimal optimizer is {best_hps.get('optimizer')}.")
print(best_hps.values)




Trial 5 Complete [00h 02m 01s]
val_accuracy: 0.2666666805744171

Best val_accuracy So Far: 0.6666666865348816
Total elapsed time: 00h 10m 55s
The optimal number of layers is 2.
The optimal filters per layer are 32, 32.
The optimal activation functions per layer are relu, relu.
The optimal learning rate for the optimizer is 0.0005.
The optimal optimizer is adam.
{'num_layers': 2, 'filters_0': 32, 'activation_0': 'relu', 'filters_1': 32, 'activation_1': 'relu', 'optimizer': 'adam', 'learning_rate': 0.0005}


In [33]:
# Import necessary libraries and modules
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam, Adadelta, Adamax
from tensorflow.keras.callbacks import EarlyStopping
from kerastuner import HyperModel, HyperParameters, Tuner
from keras_tuner.tuners import GridSearch
from tensorflow.keras.layers import Activation

# Set up image dimensions
img_height, img_width = 224, 224

# Set up the directories
train_dir = '/home/islamsa/Documents/OwlDetection/train'
validation_dir = '/home/islamsa/Documents/OwlDetection/valid'
test_dir = '/home/islamsa/Documents/OwlDetection/test'

# Data augmentation configuration for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Note: validation and test data should not be augmented
test_datagen = ImageDataGenerator(rescale=1./255)

# Data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    class_mode='categorical',
    batch_size=64
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_height, img_width),
    class_mode='categorical',
    batch_size=64
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    class_mode='categorical',
    shuffle=False
)

num_classes = train_generator.num_classes

class MyHyperModel(HyperModel):
    def __init__(self, img_height, img_width, num_classes):
        self.img_height = img_height
        self.img_width = img_width
        self.num_classes = num_classes

    def build(self, hp):
        model = Sequential()

        # Optimizing the number of convolutional layers
        for i in range(hp.Int('num_layers', 2, 4)):  # Number of layers is between 2 and 4
            model.add(Conv2D(hp.Choice(f'filters_{i}', [32, 64, 128, 256]),
                             (3, 3), padding='same',
                             input_shape=(self.img_height, self.img_width, 3)))
            model.add(Activation(hp.Choice(f'activation_{i}', ['relu', 'sigmoid', 'tanh'])))
            model.add(MaxPooling2D(pool_size=(2, 2)))
            model.add(Dropout(0.5))

        model.add(Flatten())

        model.add(Dense(256, kernel_regularizer=regularizers.l2(0.01)))
        model.add(Activation('relu'))
        model.add(Dropout(0.5))
        model.add(Dense(self.num_classes, activation='softmax'))

        # Tuner for optimizer
        hp_optimizer = hp.Choice('optimizer', ['adam', 'adadelta', 'adamax'])
        if hp_optimizer == 'adam':
            optimizer = Adam(learning_rate=hp.Choice('learning_rate', [0.001, 0.0005, 0.0001, 0.00001]))
        elif hp_optimizer == 'adadelta':
            optimizer = Adadelta(learning_rate=hp.Choice('learning_rate', [0.001, 0.0005, 0.0001, 0.00001]))
        else:  # hp_optimizer == 'adamax'
            optimizer = Adamax(learning_rate=hp.Choice('learning_rate', [0.001, 0.0005, 0.0001, 0.00001]))

        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

        return model

hypermodel = MyHyperModel(img_height, img_width, num_classes)

tuner = GridSearch(
    hypermodel,
    objective='val_loss',
    max_trials=5,  # Adjust number of trials
    executions_per_trial=1,
    directory='/home/islamsa/Documents/OwlDetection/hyperparameter_tuning_val_loss',
    project_name='owl_detection_tuning'
)

# Define the EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Start search
tuner.search(
    train_generator,
    epochs=150,
    validation_data=validation_generator,
    callbacks=[early_stopping],  # Early stopping callback
    verbose=1  # Detailed logs for each trial
)

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters()[0]

print(f"The optimal number of layers is {best_hps.get('num_layers')}.")
print(f"The optimal filters per layer are {', '.join(str(best_hps.get(f'filters_{i}')) for i in range(best_hps.get('num_layers')))}.")

# Check if each layer exists before accessing its activation function
activations = []
for i in range(best_hps.get('num_layers')):
    activation_i = best_hps.get(f'activation_{i}')
    if activation_i is not None:
        activations.append(activation_i)
print(f"The optimal activation functions per layer are {', '.join(activations)}.")

print(f"The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.")

print(f"The optimal optimizer is {best_hps.get('optimizer')}.")
print(best_hps.values)





Trial 5 Complete [00h 15m 26s]
val_loss: 6.579392910003662

Best val_loss So Far: 1.1059889793395996
Total elapsed time: 01h 00m 36s
The optimal number of layers is 2.
The optimal filters per layer are 32, 32.
The optimal activation functions per layer are relu, relu.
The optimal learning rate for the optimizer is 0.0005.
The optimal optimizer is adam.
{'num_layers': 2, 'filters_0': 32, 'activation_0': 'relu', 'filters_1': 32, 'activation_1': 'relu', 'optimizer': 'adam', 'learning_rate': 0.0005}
